In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=00632ceaf1bf65efcf036d2e4fe431cf691ed94a4df8cd9ca20abec6abe4d095
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch

import itertools
import string

import numpy as np

from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer

Загружу предобработанный датасет (с исправленной орфографией, удаленными "лишними" символами..)


In [ ]:
df_2014 = pd.read_csv('//content/drive/MyDrive/dataset_2014_corrected.csv')
df_2014 = df_2014.rename(columns={'docTypeByOP':'art',
                                  'textOCR':'text'} )
df_2014

,art,text
0,Договор,договор между российской федерацией и республи...
1,Постановление,правительство забайкальского края постановлени...
2,Распоряжение,правительство российской федерации распоряжени...
3,Постановление,а1истрация тамбовской области постановление ек...
4,Постановление,правительство рязанской области постановление ...
...,...,...
14516,Указ,указ президента российской федерации об атаман...
14517,Постановление,пр .итепьство как и про опьна 0 1801000 04 1 п...
14518,Распоряжение,правительство российской федерации распоряжени...
14519,Постановление,смоленская областная дума постановление от йй7...


In [ ]:
texts = df_2014['text'][:50].tolist()

In [ ]:
texts[30]

'указ губернатора брянской области от 1 декабря 2014 г. 376 г брянск о снятии карантина по бешенству животных на территории домовладения 28 по ул зер невской пос. туземка суземского района в связи с истечением срока наложения карантина установленного указом губернатора брянской области от 30 сентября 2014 года 305 об установлении карантина по бешенству животных на территории дома владения 28 по ул зерновкой пос. туземка суземского района и проведением мероприятий по ликвидации и профилактике бешенства согласно ветеринарным правилам от 18 июня 1996 года в 13.3. 110396 утвержденным департаментом ветеринарии министерства сельского хозяйства и продовольствия российской федерации руководствуясь законом российской федерации от 14 мая 1993 года 4979 1 о ветеринарии постановляю 1. снять карантин по бешенству животных на территории домов а дения 28 по ул зерновкой пос. туземка суземского района установ ленный указом губернатора брянской области от 30 сентября 2014 года 305 об установлении каран

In [ ]:
tokenizer = WordPunctTokenizer()
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
print(tokenizer.tokenize(texts[30]))

['указ', 'губернатора', 'брянской', 'области', 'от', '1', 'декабря', '2014', 'г', '.', '376', 'г', 'брянск', 'о', 'снятии', 'карантина', 'по', 'бешенству', 'животных', 'на', 'территории', 'домовладения', '28', 'по', 'ул', 'зер', 'невской', 'пос', '.', 'туземка', 'суземского', 'района', 'в', 'связи', 'с', 'истечением', 'срока', 'наложения', 'карантина', 'установленного', 'указом', 'губернатора', 'брянской', 'области', 'от', '30', 'сентября', '2014', 'года', '305', 'об', 'установлении', 'карантина', 'по', 'бешенству', 'животных', 'на', 'территории', 'дома', 'владения', '28', 'по', 'ул', 'зерновкой', 'пос', '.', 'туземка', 'суземского', 'района', 'и', 'проведением', 'мероприятий', 'по', 'ликвидации', 'и', 'профилактике', 'бешенства', 'согласно', 'ветеринарным', 'правилам', 'от', '18', 'июня', '1996', 'года', 'в', '13', '.', '3', '.', '110396', 'утвержденным', 'департаментом', 'ветеринарии', 'министерства', 'сельского', 'хозяйства', 'и', 'продовольствия', 'российской', 'федерации', 'руково

Фунция для токенизации и лемматизации текстов докуметов


In [ ]:
def tokenize_lemmatize(text):
    text_tokens = tokenizer.tokenize(text)
    res = list()
    for word in text_tokens:
        p = morph.parse(word)[0]
        if len(p.normal_form)>3:
          res.append(p.normal_form)

    return ' '.join(res)

print(tokenize_lemmatize(texts[30]))

указ губернатор брянский область декабрь 2014 брянск снятие карантин бешенство животное территория домовладение невский туземка суземский район связь истечение срок наложение карантин установленный указ губернатор брянский область сентябрь 2014 установление карантин бешенство животное территория владение зерновка туземка суземский район проведение мероприятие ликвидация профилактика бешенство согласно ветеринарный правило июнь 1996 110396 утвердить департамент ветеринария министерство сельский хозяйство продовольствие российский федерация руководствоваться закон российский федерация 1993 4979 ветеринария постановлять снять карантин бешенство животное территория дения зерновка туземка суземский район устан ленный указ губернатор брянский область сентябрь 2014 установление карантин бешенство животное территория домовладение зерновка туземка суземский район указ губернатор брянский область сентябрь 2014 установление карантин бешенство животное территория домовладение зерновка туземка сузе

In [ ]:
df_2014['art'].value_counts()
df_2014['art'].unique().tolist()

['Договор',
 'Постановление',
 'Распоряжение',
 'Областной закон',
 'Указ',
 'Закон',
 'Соглашение',
 'Федеральный закон',
 'Приказ',
 'Определение',
 'Федеральный конституционный закон',
 'Протокол',
 'Официальное сообщение',
 'Закон Российской Федерации о поправке к Конституции Российской Федерации',
 'Уставный закон',
 'Исполнительный протокол',
 'Поправки к Уставу',
 'Сообщение',
 'Конвенция']

Поскольку классы не сбалансированны, возьму только 5 наиболее популярных и буду проводить классификацию на них

In [ ]:
df_2014 = df_2014[df_2014['art'].isin(['Постановление', 'Распоряжение', 'Закон', 'Указ', 'Федеральный закон'])]
target_names = list(set(df_2014['art'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)
df_2014['art'] = df_2014['art'].replace(label2idx)

{'Закон': 0, 'Федеральный закон': 1, 'Постановление': 2, 'Распоряжение': 3, 'Указ': 4}


<ipython-input-11-ec8d66952e1c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['art'] = df_2014['art'].replace(label2idx)


In [ ]:
df_2014['text_token'] = df_2014['text'].apply(tokenize_lemmatize)

<ipython-input-12-ac75249c9ac0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['text_token'] = df_2014['text'].apply(tokenize_lemmatize)


In [ ]:
df_2014

,art,text,text_token
1,2,правительство забайкальского края постановлени...,правительство забайкальский край постановление...
2,3,правительство российской федерации распоряжени...,правительство российский федерация распоряжени...
3,2,а1истрация тамбовской области постановление ек...,а1истрация тамбовский область постановление та...
4,2,правительство рязанской области постановление ...,правительство рязанский область постановление ...
5,3,правительство российской федерацш распоряжение...,правительство российский федерацш распоряжение...
...,...,...,...
14516,4,указ президента российской федерации об атаман...,указ президент российский федерация атаман заб...
14517,2,пр .итепьство как и про опьна 0 1801000 04 1 п...,итепьство опьный 1801000 постановление правите...
14518,3,правительство российской федерации распоряжени...,правительство российский федерация распоряжени...
14519,2,смоленская областная дума постановление от йй7...,смоленский областной дума постановление йй7дмт...


Для того, чтобы в будущем не ждать результата работы токенизирующей функции, сохраню данные в данном виде


In [ ]:
#/content/drive/MyDrive
df_2014.to_csv('/content/drive/MyDrive/df_2014_token_lemm.csv', index=False)

In [5]:
df_2014 = pd.read_csv('//content/drive/MyDrive/df_2014_token_lemm.csv')

In [6]:
import gensim.downloader as api
print("\n".join(api.info()['models'].keys()))


fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
word2vec = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


In [8]:
tok = df_2014['text_token'][:3].str.split().tolist()


Посмотрим на пример работы word2vec

In [9]:
word2vec.get_vector(tok[1][2]), tok[1][2], word2vec.get_vector(tok[1][2]).size

(array([ 6.0851e-01, -1.6958e-01, -4.2273e-01,  1.7424e-01, -1.4925e-01,
        -7.7039e-01,  6.0080e-02, -2.4123e-01,  6.3602e-01, -2.2287e-01,
        -1.8949e-01,  4.9296e-01,  6.0794e-01, -3.5787e-02,  1.3970e+00,
        -9.8514e-02,  5.3052e-01,  8.9576e-01,  1.4661e+00,  4.6343e-02,
        -6.7487e-01,  6.8185e-01, -6.1941e-01, -1.3757e+00,  1.3445e+00,
         1.2998e+00, -4.3413e-02, -4.4610e-01, -3.1541e-01, -2.0677e-01,
         6.4849e-01,  1.8429e-01, -1.0840e+00, -6.6575e-02, -9.3601e-01,
         3.4869e-01,  1.2554e-03,  5.8665e-01,  1.5468e-01, -2.3753e-01,
         5.8895e-01,  8.1119e-01,  3.8179e-01,  5.0512e-02,  5.2445e-01,
         2.5004e-01,  4.3421e-02, -5.5800e-03, -4.5384e-01,  1.9952e+00],
       dtype=float32),
 'федерация',
 50)

In [10]:
from torch.utils.data import Dataset, random_split
from torch.utils.data import DataLoader

In [11]:
import random

SEED = 42
RANDOM_STATE = SEED
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True


train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device



CUDA is available!  Training on GPU ...


device(type='cuda', index=0)

In [12]:
class MyDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: word2vec):

        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec


        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.data[self.target_column][item]


        embeddings = self.get_embeddings_([w for w in text.split()])

        return {"feature": embeddings, "target": label}



    def get_embeddings_(self, tokens):

        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [13]:
import math
indexes = np.arange(df_2014.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(df_2014.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = df_2014.iloc[dev_indexes]
test_data = df_2014.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [14]:
dev = MyDataset(dev_data, "text_token", "art", word2vec)

In [15]:
batch_size = 256

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch])

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          collate_fn=average_emb)

valid_loader = DataLoader(valid,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False,
                          collate_fn=average_emb)

In [16]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
import torch.nn as nn
from torchsummary import summary

In [17]:
def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    running_loss = 0
    model.train()

    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        optimizer.zero_grad()


        outp = model(features)


        loss = criterion(outp, targets.long()) # лосс
        running_loss += loss.item()


        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {running_loss/len(train_loader):.4}")



def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)


            outp = model(features)

            loss = criterion(outp, targets.long()) # лосс

            y_pred = torch.softmax(outp, dim=1).argmax(dim=1)

            acc = ((y_pred==targets).detach().cpu().numpy().mean())
            print(y_pred[:10], targets[:10])


            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [18]:
vector_size = dev.word2vec.vector_size
num_classes = 5
lr = 1e-2
num_epochs = 20

class SimpleNet(nn.Module):
    def __init__(self, nX,  nY):
        super(SimpleNet, self).__init__()

        self.fc1 = nn.Sequential(
            nn.Linear(nX, 25),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(25, 25),
            nn.ReLU()
        )

        self.fc3 = nn.Sequential(
            nn.Linear(25, nY),


        )

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = SimpleNet(vector_size,  5) # модель

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

In [19]:
from torchsummary import summary
summary(model,(vector_size,vector_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 50, 25]           1,275
              ReLU-2               [-1, 50, 25]               0
            Linear-3               [-1, 50, 25]             650
              ReLU-4               [-1, 50, 25]               0
            Linear-5                [-1, 50, 5]             130
Total params: 2,055
Trainable params: 2,055
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [20]:
from tqdm.notebook import tqdm

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([3., 2., 2., 2., 0., 0., 4., 2., 2., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 3., 1., 2., 3., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 3., 2., 4., 2., 3., 2., 2., 2., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([3., 0., 0., 0., 3., 2., 2., 3., 3., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 4., 2., 2., 2., 4., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([0., 2., 2., 2., 2., 2., 3., 3., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 2., 2., 4., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 3., 2., 1., 4., 2., 4., 2., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2

Epoch 2. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([3, 2, 2, 2, 0, 0, 2, 2, 2, 3], device='cuda:0') tensor([3., 2., 2., 2., 0., 0., 4., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 1, 2, 3, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 3., 1., 2., 3., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 3, 3, 2, 2, 3, 2, 2, 2, 3], device='cuda:0') tensor([2., 3., 2., 4., 2., 3., 2., 2., 2., 3.], device='cuda:0')
tensor([3, 0, 0, 0, 2, 2, 2, 3, 3, 2], device='cuda:0') tensor([3., 0., 0., 0., 3., 2., 2., 3., 3., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 3, 3, 2], device='cuda:0') tensor([2., 2., 2., 4., 2., 2., 2., 4., 2., 2.], device='cuda:0')
tensor([0, 2, 2, 2, 2, 2, 3, 3, 2, 2], device='cuda:0') tensor([0., 2., 2., 2., 2., 2., 3., 3., 2., 2.], device='cuda:0')
tensor([2, 2, 3, 2, 2, 3, 2, 2, 4, 3], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 2., 2., 4., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 1, 3, 2, 0, 2, 3], device='cuda:0') tensor([2., 2., 3., 2., 1., 4., 2., 4., 2., 3.], device='cuda:0')
tensor([3, 2, 2, 0, 2, 0

Epoch 20. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([3, 2, 2, 3, 0, 0, 2, 2, 2, 3], device='cuda:0') tensor([3., 2., 2., 2., 0., 0., 4., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 1, 2, 3, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 3., 1., 2., 3., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 3, 3, 2, 2, 3, 2, 2, 2, 3], device='cuda:0') tensor([2., 3., 2., 4., 2., 3., 2., 2., 2., 3.], device='cuda:0')
tensor([3, 0, 0, 0, 2, 2, 2, 3, 4, 3], device='cuda:0') tensor([3., 0., 0., 0., 3., 2., 2., 3., 3., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 3, 3, 2], device='cuda:0') tensor([2., 2., 2., 4., 2., 2., 2., 4., 2., 2.], device='cuda:0')
tensor([0, 2, 2, 2, 2, 2, 3, 3, 2, 2], device='cuda:0') tensor([0., 2., 2., 2., 2., 2., 3., 3., 2., 2.], device='cuda:0')
tensor([2, 2, 3, 2, 2, 3, 2, 2, 4, 3], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 2., 2., 4., 3.], device='cuda:0')
tensor([2, 2, 2, 2, 1, 4, 2, 4, 2, 3], device='cuda:0') tensor([2., 2., 3., 2., 1., 4., 2., 4., 2., 3.], device='cuda:0')
tensor([3, 2, 2, 0, 3, 0

In [21]:
test_loader = DataLoader(
    MyDataset(test_data, "text_token", "art", word2vec),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/12 [00:00<?, ?it/s]

tensor([2, 4, 0, 2, 2, 2, 1, 2, 2, 3], device='cuda:0') tensor([2., 4., 0., 2., 2., 2., 1., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 3, 3, 2, 2, 0, 2, 3, 2], device='cuda:0') tensor([2., 3., 3., 3., 2., 2., 0., 2., 4., 2.], device='cuda:0')
tensor([2, 2, 3, 4, 2, 2, 0, 2, 4, 4], device='cuda:0') tensor([2., 2., 3., 4., 3., 3., 0., 2., 4., 4.], device='cuda:0')
tensor([0, 3, 1, 2, 0, 2, 3, 2, 2, 2], device='cuda:0') tensor([0., 2., 1., 2., 0., 2., 4., 2., 2., 2.], device='cuda:0')
tensor([0, 2, 0, 2, 2, 2, 2, 3, 2, 2], device='cuda:0') tensor([0., 2., 0., 2., 2., 2., 2., 3., 2., 2.], device='cuda:0')
tensor([2, 4, 3, 2, 3, 4, 2, 2, 2, 2], device='cuda:0') tensor([2., 4., 3., 4., 3., 4., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 0, 2, 2, 2, 2, 2, 1, 2, 2], device='cuda:0') tensor([2., 0., 2., 2., 2., 2., 2., 1., 2., 2.], device='cuda:0')
tensor([3, 2, 1, 1, 4, 3, 2, 2, 2, 2], device='cuda:0') tensor([3., 2., 1., 1., 2., 3., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 2, 3, 0, 4, 3

А если умножить эмбеддинги слов на соответствующий им TfidfVectorizer?

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf  = TfidfVectorizer()
tfidf.fit([' '.join(i) for i in df_2014['text_token'].str.split()])

TfidfVectorizer()

In [23]:
tfidf.get_feature_names_out()

array(['0000', '00000', '00000000', ..., 'ёмкий', 'ёмкость', 'ёрник'],
      dtype=object)

In [24]:
tf_dict=dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))
tf_dict['федерация']

1.5072087284706561

In [25]:
class MyDataset_2(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: word2vec, tfidf: tf_dict):

        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tf_dict = tfidf

        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.data[self.target_column][item]


        embeddings = self.get_embeddings_([w for w in text.split()])
        vecs = self.get_vecs_([w for w in text.split()])

        return {"feature": embeddings, 'vecs':vecs, "target": label}



    def get_embeddings_(self, tokens):

        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_vecs_(self, tokens):

        vecs = [self.tf_dict[w] for w in tokens if w in self.word2vec]

        return vecs

    def __len__(self):
        return self.data.shape[0]



In [26]:
dev = MyDataset_2(dev_data, "text_token", "art", word2vec,tf_dict)

In [38]:
batch_size = 256

def average_emb(batch):

    features = np.array([np.mean(b["feature"].dot(b['vecs'][0]), axis=0) for b in batch])

    targets = np.array([b["target"] for b in batch])



    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}




train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True,
                          collate_fn=average_emb)

valid_loader = DataLoader(valid,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False,
                          collate_fn=average_emb)

In [39]:
model = SimpleNet(vector_size,  5) # модель

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

In [40]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([2, 2, 2, 2, 2, 3, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 0., 2., 0., 3., 3., 2., 2., 2.], device='cuda:0')
tensor([2, 4, 4, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([4., 4., 4., 2., 2., 3., 2., 3., 2., 2.], device='cuda:0')
tensor([2, 3, 4, 3, 2, 2, 2, 3, 2, 2], device='cuda:0') tensor([3., 4., 2., 4., 2., 2., 3., 3., 0., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 0, 3, 2, 3, 3], device='cuda:0') tensor([3., 2., 2., 2., 2., 0., 3., 2., 2., 2.], device='cuda:0')
tensor([2, 3, 2, 3, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 3., 3., 3., 0., 2., 2., 3., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 3, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 3., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 0, 2, 3, 2, 3, 2, 2, 2], device='cuda:0') tensor([4., 3., 2., 2., 3., 2., 4., 2., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2

Epoch 2. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([2, 2, 0, 0, 0, 3, 3, 2, 2, 2], device='cuda:0') tensor([2., 2., 0., 2., 0., 3., 3., 2., 2., 2.], device='cuda:0')
tensor([2, 4, 4, 2, 2, 3, 2, 3, 3, 2], device='cuda:0') tensor([4., 4., 4., 2., 2., 3., 2., 3., 2., 2.], device='cuda:0')
tensor([3, 4, 2, 4, 2, 2, 3, 3, 0, 2], device='cuda:0') tensor([3., 4., 2., 4., 2., 2., 3., 3., 0., 2.], device='cuda:0')
tensor([3, 2, 2, 4, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([3, 3, 2, 2, 2, 0, 3, 2, 2, 3], device='cuda:0') tensor([3., 2., 2., 2., 2., 0., 3., 2., 2., 2.], device='cuda:0')
tensor([3, 3, 2, 3, 3, 0, 2, 2, 3, 2], device='cuda:0') tensor([2., 2., 3., 3., 3., 0., 2., 2., 3., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 3, 2, 2, 3], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 3., 2., 2., 3.], device='cuda:0')
tensor([4, 3, 2, 2, 4, 2, 4, 2, 2, 2], device='cuda:0') tensor([4., 3., 2., 2., 3., 2., 4., 2., 2., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 2

Epoch 20. Train Loss: 0:   0%|          | 0/35 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/9 [00:00<?, ?it/s]

tensor([2, 2, 0, 0, 0, 3, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 0., 2., 0., 3., 3., 2., 2., 2.], device='cuda:0')
tensor([2, 4, 4, 2, 2, 3, 2, 3, 2, 2], device='cuda:0') tensor([4., 4., 4., 2., 2., 3., 2., 3., 2., 2.], device='cuda:0')
tensor([3, 4, 2, 4, 2, 2, 2, 3, 0, 2], device='cuda:0') tensor([3., 4., 2., 4., 2., 2., 3., 3., 0., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 0, 3, 2, 2, 3], device='cuda:0') tensor([3., 2., 2., 2., 2., 0., 3., 2., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 3, 3, 0, 2, 2, 2, 2], device='cuda:0') tensor([2., 2., 3., 3., 3., 0., 2., 2., 3., 2.], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 3], device='cuda:0') tensor([2., 2., 2., 2., 2., 3., 3., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 2, 2, 2, 2, 4, 2, 2, 2], device='cuda:0') tensor([4., 3., 2., 2., 3., 2., 4., 2., 2., 2.], device='cuda:0')
tensor([3, 2, 2, 2, 2, 2

In [41]:
test_loader = DataLoader(
    MyDataset_2(test_data, "text_token", "art", word2vec, tf_dict),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/12 [00:00<?, ?it/s]

tensor([2, 4, 0, 2, 2, 2, 1, 2, 2, 3], device='cuda:0') tensor([2., 4., 0., 2., 2., 2., 1., 2., 2., 3.], device='cuda:0')
tensor([2, 3, 3, 3, 2, 2, 0, 2, 4, 2], device='cuda:0') tensor([2., 3., 3., 3., 2., 2., 0., 2., 4., 2.], device='cuda:0')
tensor([2, 2, 3, 4, 2, 3, 0, 2, 4, 4], device='cuda:0') tensor([2., 2., 3., 4., 3., 3., 0., 2., 4., 4.], device='cuda:0')
tensor([0, 3, 1, 2, 0, 2, 3, 2, 2, 3], device='cuda:0') tensor([0., 2., 1., 2., 0., 2., 4., 2., 2., 2.], device='cuda:0')
tensor([0, 2, 0, 2, 2, 2, 2, 3, 2, 2], device='cuda:0') tensor([0., 2., 0., 2., 2., 2., 2., 3., 2., 2.], device='cuda:0')
tensor([2, 4, 3, 2, 3, 4, 2, 2, 2, 2], device='cuda:0') tensor([2., 4., 3., 4., 3., 4., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 0, 2, 2, 2, 2, 2, 1, 2, 2], device='cuda:0') tensor([2., 0., 2., 2., 2., 2., 2., 1., 2., 2.], device='cuda:0')
tensor([3, 2, 1, 1, 4, 3, 2, 2, 2, 2], device='cuda:0') tensor([3., 2., 1., 1., 2., 3., 2., 2., 2., 2.], device='cuda:0')
tensor([2, 2, 2, 0, 4, 3